In [21]:
''' This cell contains all constants thay may differ on our machines '''

ELASTIC_HOST = 'localhost'
ELASTIC_PORT = 9200
COLLECTION_DIRECTORY = "../byweb" # directory with .out files to process
COLLECTION_DIRECTORY_MYSTEM = "../byweb_stem" # directory with .out files after mystem processing

QUERIES_FILE = "web2008_adhoc.xml"
RELEVANCE_FILE = "relevant_table_2009.xml"

In [4]:
%config IPCompleter.greedy=True
import re
import json
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
import requests
from time import time

In [ ]:
es = Elasticsearch([{'host': ELASTIC_HOST, 'port': ELASTIC_PORT, 'timeout': 360, 'maxsize': 25}])

In [ ]:
settings = {
    'mappings': {
        'properties': {
            'content': {
                'type': 'text'
            }
        }
    }
}

In [ ]:
def recreate_index():
    es.indices.delete(index='hw2index')
    es.indices.create(index='hw2index', body=settings)

In [ ]:
recreate_index()

In [ ]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

In [24]:
class Document:
    def __init__(self, doc_url, doc_id, sz_bytes, sz_words):
        self.url = doc_url       # document url
        self.id = doc_id         # unique document id (str)
        self.sz_bytes = sz_bytes # document size in bytes before deleting html markup
        self.sz_words = sz_words # number of words in document before deleting html markup
        self.words = []          # list of words in document after deleting html markup
        self.links = []          # lisk of links in document

In [25]:
import os

In [26]:
from tqdm import tqdm
from tqdm import tqdm_notebook
import pickle

class BaseDocumentProcessor:
    def process(self, document):
        pass
    def result(self):
        pass

def process_file(d, f, processor, pbar):
    print("processing", os.path.join(d, f))
    with open(os.path.join(d, f), "rb") as fin:
        while True:
            pbar.update(1)
            try:
                document = pickle.load(fin)
            except:
                break
            processor.process(document)

def process_collection(directory, processor):
    pbar = tqdm(total = 200000)
    for file in os.listdir(directory):
        if (file.endswith(".out")):
            process_file(directory, file, processor, pbar)

In [ ]:

class IndexDocs(BaseDocumentProcessor):
    def __init__(self):
        """ do all initialization here """
        self.actions = []
    
    def process(self, document):
        """ document: Document (see first cell)
            process each document here """
        #print(json.dumps({'content' : document.words}))
        self.actions.append(create_es_action('hw2index', document.id, json.dumps({'content' : document.words})))
        
    def result(self):
        return self.actions
        


In [ ]:
def es_actions_generator():
    processor = IndexDocs()        
    process_collection(COLLECTION_DIRECTORY, processor)
    return processor.result()

In [ ]:
import time

In [ ]:
start = time.time()
for ok, result in tqdm_notebook(parallel_bulk(es, es_actions_generator(), queue_size=4, thread_count=4, chunk_size=1000)):
    if not ok:
        print(result)
end = time.time()
print('Time=' + str(end - start))


In [ ]:
import requests
param = (('v', ''),) # '-v' is for --verbose

# call the class's method to get an HTTP response model
resp = requests.get(f'http://{ELASTIC_HOST}:{ELASTIC_PORT}/_cat/indices', params=param)


In [ ]:
resp.text

In [ ]:
settings = {
    'mappings': {
        'properties': {
            'content': {
                'type': 'text'
            }
        }
    },
    "settings": {
        "analysis" : {
            "analyzer" : {
                "my_analyzer" : {
                    "tokenizer" : "standard",
                    "filter" : ["lowercase", "my_snow"]
                }
            },
            "filter" : {
                "my_snow" : {
                    "type" : "snowball",
                    "language" : "russian"
                }
            }
        }
    }
}

In [ ]:
recreate_index()

In [ ]:
start = time.time()
for ok, result in tqdm_notebook(parallel_bulk(es, es_actions_generator(), queue_size=4, thread_count=4, chunk_size=1000)):
    if not ok:
        print(result)
end = time.time()
print('Time=' + str(end - start))



In [ ]:
requests.get(f'http://{ELASTIC_HOST}:{ELASTIC_PORT}/_cat/indices', params=param).text

In [ ]:
from bs4 import BeautifulSoup

def read_queries():
    queries = {}
    with open(QUERIES_FILE, "rt", encoding="cp1251") as file:
        bs = BeautifulSoup(file.read())
        for task in bs.find_all("task"):
            queries[task["id"]] = task.text
    return queries

def read_relevance():
    relevance = {}
    with open(RELEVANCE_FILE, "rt", encoding="cp1251") as file:
        bs = BeautifulSoup(file.read())
        for task in bs.find_all("task"):
            rel = [doc["id"] for doc in task.find_all("document") if doc["relevance"] == "vital"]
            if rel:
                relevance[task["id"]] = rel
    return relevance

In [ ]:
relevance = read_relevance()
queries = read_queries()

In [ ]:
def build_query(query):
    return { 'query': { 'bool': { 'should': [ { 'match': { 'content': query } } ] } } }

def run_search(query, size):
    search_result = es.search(index='hw2index', body=build_query(query), size=size)['hits']
    return [hit['_id'] for hit in search_result['hits']]

def count_metrics():
    p = 0
    r = 0
    rp = 0
    ma_p = 0
    n = 0
    for qid, query in queries.items():
        if qid not in relevance.keys():
            continue
        n += 1
        rel = relevance[qid]
        relevant = len(rel)
        hits = run_search(query, max(20, relevant))
        hits_r = hits[:relevant]
        hits_20 = hits[:20]
        true_positive_20 = sum(1 for did in hits_20 if did in rel)
        true_positive_r = sum(1 for did in hits_r if did in rel)
        p += true_positive_20 / 20
        r += true_positive_20 / relevant
        rp += true_positive_r / relevant
        ma_p += sum(sum(1 for did in hits[:k] if did in rel) / k for k in range(1, 21)) / 20
    print("p@20:", p / n)
    print("r@20:", r / n)
    print("r-precision:", rp / n)
    print("map@20:", ma_p / n)

In [ ]:
count_metrics()

In [31]:
class MystemProcessor(BaseDocumentProcessor):
    def __init__(self, fout):
        self.m = Mystem(grammar_info=False, disambiguation=False)
        reg = re.compile('[a-zа-яё0-9\-]')
        self.filterFunc = lambda w : reg.match(w)
        self.fout = fout

    def process(self, document):
        text = ' '.join(document.words).lower()
        lemmas = self.m.lemmatize(text)
        words = list(filter(self.filterFunc, lemmas))
        doc = Document(document.url, document.id, document.sz_bytes, document.sz_words)
        doc.words = words
        doc.links = document.links
        pickle.dump(doc, self.fout)

         
    def result(self):
        pass

def process_collection_with_mystem(directory, outdirectory):
    pbar = tqdm(total = 200000)
    for file in os.listdir(directory):
        if (file.endswith(".out")):        
            with open(os.path.join(outdirectory, file), "wb") as fout:
                processor = MystemProcessor(fout)
                process_file(directory, file, processor, pbar)

In [32]:
process_collection_with_mystem(COLLECTION_DIRECTORY, COLLECTION_DIRECTORY_MYSTEM)






  0%|                                                                                       | 0/200000 [00:00<?, ?it/s]

processing ..\byweb\byweb.0.out







  0%|                                                                             | 3/200000 [00:00<3:03:58, 18.12it/s]




  0%|                                                                             | 4/200000 [00:00<5:51:13,  9.49it/s]




  0%|                                                                             | 6/200000 [00:00<5:43:23,  9.71it/s]




  0%|                                                                             | 8/200000 [00:00<6:53:27,  8.06it/s]




  0%|                                                                             | 9/200000 [00:01<9:32:10,  5.83it/s]




  0%|                                                                            | 11/200000 [00:01<8:01:02,  6.93it/s]




  0%|                                                                            | 12/200000 [00:01<7:40:37,  7.24it/s]




  0%|                                                                            | 13/200000 [00:01<7:54:54,  7.02it/s]




  0

  0%|                                                                           | 114/200000 [00:12<4:57:25, 11.20it/s]




  0%|                                                                           | 116/200000 [00:12<4:48:03, 11.56it/s]




  0%|                                                                           | 118/200000 [00:12<5:29:21, 10.11it/s]




  0%|                                                                           | 120/200000 [00:12<5:23:01, 10.31it/s]




  0%|                                                                           | 122/200000 [00:12<5:08:51, 10.79it/s]




  0%|                                                                           | 124/200000 [00:13<4:47:39, 11.58it/s]




  0%|                                                                           | 126/200000 [00:13<4:55:54, 11.26it/s]




  0%|                                                                           | 128/200000 [00:13<4:42:09, 11.81it/s]




  0%|   

  0%|                                                                           | 240/200000 [00:25<7:14:37,  7.66it/s]




  0%|                                                                           | 241/200000 [00:25<6:51:49,  8.08it/s]




  0%|                                                                           | 242/200000 [00:25<6:40:29,  8.31it/s]




  0%|                                                                           | 243/200000 [00:25<6:22:13,  8.71it/s]




  0%|                                                                           | 244/200000 [00:25<6:21:56,  8.72it/s]




  0%|                                                                           | 246/200000 [00:26<5:44:38,  9.66it/s]




  0%|                                                                           | 248/200000 [00:26<5:21:34, 10.35it/s]




  0%|                                                                           | 250/200000 [00:26<5:13:03, 10.63it/s]




  0%|   

...

  1%|▋                                                                         | 1979/200000 [03:14<5:04:15, 10.85it/s]




  1%|▋                                                                         | 1981/200000 [03:14<4:59:04, 11.03it/s]




  1%|▋                                                                         | 1983/200000 [03:15<9:10:57,  5.99it/s]




  1%|▋                                                                         | 1985/200000 [03:15<8:24:58,  6.54it/s]




  1%|▋                                                                         | 1987/200000 [03:15<7:07:18,  7.72it/s]




  1%|▋                                                                         | 1989/200000 [03:15<6:18:06,  8.73it/s]




  1%|▋                                                                         | 1991/200000 [03:16<5:40:48,  9.68it/s]




  1%|▋                                                                         | 1993/200000 [03:16<5:17:06, 10.41it/s]




  1%|▋  

  1%|▊                                                                         | 2104/200000 [03:27<6:18:46,  8.71it/s]




  1%|▊                                                                         | 2106/200000 [03:27<5:44:14,  9.58it/s]




  1%|▊                                                                         | 2108/200000 [03:27<5:19:09, 10.33it/s]




  1%|▊                                                                         | 2110/200000 [03:27<5:08:20, 10.70it/s]




  1%|▊                                                                         | 2112/200000 [03:28<7:10:29,  7.66it/s]




  1%|▊                                                                         | 2114/200000 [03:28<7:08:52,  7.69it/s]




  1%|▊                                                                         | 2116/200000 [03:28<6:44:31,  8.15it/s]




  1%|▊                                                                         | 2118/200000 [03:28<6:08:12,  8.96it/s]




  1%|▊  

  1%|▊                                                                         | 2229/200000 [03:39<4:20:00, 12.68it/s]




  1%|▊                                                                         | 2231/200000 [03:39<4:11:51, 13.09it/s]




  1%|▊                                                                         | 2233/200000 [03:39<4:11:58, 13.08it/s]




  1%|▊                                                                         | 2235/200000 [03:39<4:11:52, 13.09it/s]




  1%|▊                                                                         | 2237/200000 [03:39<4:01:41, 13.64it/s]




  1%|▊                                                                         | 2239/200000 [03:39<4:40:51, 11.74it/s]




  1%|▊                                                                         | 2241/200000 [03:40<4:50:18, 11.35it/s]




  1%|▊                                                                         | 2243/200000 [03:40<4:36:41, 11.91it/s]




  1%|▊  

  1%|▊                                                                         | 2344/200000 [03:50<8:18:24,  6.61it/s]




  1%|▊                                                                         | 2345/200000 [03:51<7:57:29,  6.90it/s]




  1%|▊                                                                         | 2346/200000 [03:51<7:26:31,  7.38it/s]




  1%|▊                                                                         | 2347/200000 [03:51<7:34:48,  7.24it/s]




  1%|▊                                                                        | 2348/200000 [03:51<14:36:59,  3.76it/s]




  1%|▊                                                                        | 2350/200000 [03:52<12:17:46,  4.46it/s]




  1%|▊                                                                         | 2352/200000 [03:52<9:50:29,  5.58it/s]




  1%|▊                                                                         | 2354/200000 [03:52<8:12:22,  6.69it/s]




  1%|▊  

KeyboardInterrupt: 